In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7de8774e9800>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error
[nltk_data] Downloading package stopwords to /root/nltk_data...

True

In [3]:
file_path = 'Apple_data.csv'
data = pd.read_csv(file_path)
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date', ascending=True)
data = data.drop(index=220)
def fill_missing_prices(data):
    data = data.reset_index(drop=True)
    for i in range(len(data)):
        if np.isnan(data.loc[i, 'Open Price']) and i > 0:
            data.loc[i, 'Open Price'] = data.loc[i - 1, 'Close Price']
        if np.isnan(data.loc[i, 'Close Price']) and i < len(data) - 1:
            data.loc[i, 'Close Price'] = data.loc[i + 1, 'Open Price']

    return data
data = fill_missing_prices(data)
data = data.dropna()
data.head()

,Date,company,headline,abstract,url,section,Open Price,Close Price
0,2022-12-20,Apple,Bans on TikTok Gain Momentum in Washington and...,At least 14 states have passed bans on the ser...,https://www.nytimes.com/2022/12/20/technology/...,Technology,130.02,130.92
1,2022-12-20,Apple,Clean Energy Quest Pits Google Against Utilities,Google says its goals for carbon-free power ar...,https://www.nytimes.com/2022/12/20/business/go...,Business Day,130.02,130.92
2,2022-12-20,Apple,Amazon and E.U. Reach Deal to End Antitrust In...,The online retail giant avoided a major fine b...,https://www.nytimes.com/2022/12/20/technology/...,Technology,130.02,130.92
3,2022-12-21,Apple,YouTube in Advanced Talks for N.F.L. Sunday Ti...,"Tech’s biggest companies — Apple, Amazon and Y...",https://www.nytimes.com/2022/12/20/business/nf...,Business Day,131.60,134.04
4,2022-12-21,Apple,A New Chat Bot Is a ‘Code Red’ for Google’s Se...,A new wave of chat bots like ChatGPT use artif...,https://www.nytimes.com/2022/12/21/technology/...,Technology,131.60,134.04


In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

data['cleaned_headline'] = data['headline'].apply(clean_text)
data['cleaned_abstract'] = data['abstract'].apply(clean_text)

def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

data['headline_sentiment'] = data['cleaned_headline'].apply(get_sentiment)
data['abstract_sentiment'] = data['cleaned_abstract'].apply(get_sentiment)

sia = SentimentIntensityAnalyzer()

data['headline_sentiment'] = data['cleaned_headline'].apply(lambda x: sia.polarity_scores(x)['compound'])
data['abstract_sentiment'] = data['cleaned_abstract'].apply(lambda x: sia.polarity_scores(x)['compound'])

vectorizer = TfidfVectorizer(max_features=1000)


headline_features = vectorizer.fit_transform(data['cleaned_headline'])
abstract_features = vectorizer.fit_transform(data['cleaned_abstract'])
data['price_change'] = (data['Close Price'] - data['Open Price']) / data['Open Price']

X = data[['headline_sentiment', 'abstract_sentiment']]
y = data['Close Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

mape = (abs((y_test - predictions) / y_test.replace(0, 1)).mean()) * 100


print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")
print(f"MAPE: {mape:.2f}%")

MAE: 23.177875072407804
MSE: 820.5922195630922
R²: 0.00029425255934123484
MAPE: 12.64%


In [5]:
sia = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    return sia.polarity_scores(text)['compound']

data['headline_sentiment'] = data['headline'].apply(get_vader_sentiment)
data['abstract_sentiment'] = data['abstract'].apply(get_vader_sentiment)
data['price_increase'] = (data['Close Price'] > data['Open Price']).astype(int)
daily_data = data.groupby('Date').agg({
    'headline_sentiment': ['mean', 'max', 'count'],
    'abstract_sentiment': ['mean', 'max', 'count'],
    'Open Price': 'first',
    'Close Price': 'last',
    'price_increase': 'last'
})

daily_data.columns = ['_'.join(col).strip() for col in daily_data.columns]
daily_data = daily_data.reset_index()
X = daily_data[['headline_sentiment_mean', 'headline_sentiment_max', 'headline_sentiment_count',
                'abstract_sentiment_mean', 'abstract_sentiment_max', 'abstract_sentiment_count']]
y = daily_data['price_increase_last']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5568181818181818
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.29      0.36        38
           1       0.58      0.76      0.66        50

    accuracy                           0.56        88
   macro avg       0.53      0.52      0.51        88
weighted avg       0.54      0.56      0.53        88

Confusion Matrix:
 [[11 27]
 [12 38]]


In [6]:
daily_data = daily_data.sort_values('Date')
daily_data['Close_Price_lag1'] = daily_data['Close Price_last'].shift(1)
daily_data['Close_Price_diff'] = daily_data['Close Price_last'] - daily_data['Close_Price_lag1']
daily_data['Close_Price_diff_pct'] = daily_data['Close_Price_diff'] / daily_data['Close_Price_lag1']
daily_data = daily_data.dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5568181818181818
              precision    recall  f1-score   support

           0       0.49      0.55      0.52        38
           1       0.62      0.56      0.59        50

    accuracy                           0.56        88
   macro avg       0.56      0.56      0.55        88
weighted avg       0.56      0.56      0.56        88

Confusion Matrix:
 [[21 17]
 [22 28]]


In [16]:
daily_data = daily_data.sort_values('Date')
daily_data['Close_Price_lag1'] = daily_data['Close Price_last'].shift(1)
daily_data['Close_Price_diff'] = daily_data['Close Price_last'] - daily_data['Close_Price_lag1']
daily_data['Close_Price_diff_pct'] = daily_data['Close_Price_diff'] / daily_data['Close_Price_lag1']
daily_data = daily_data.dropna()

X = daily_data[['headline_sentiment_mean', 'headline_sentiment_max', 'abstract_sentiment_mean', 'abstract_sentiment_max',
                'headline_sentiment_count', 'abstract_sentiment_count', 'Close_Price_diff', 'Close_Price_diff_pct']]
y = daily_data['price_increase_last']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8409090909090909
              precision    recall  f1-score   support

           0       0.78      0.86      0.82        36
           1       0.90      0.83      0.86        52

    accuracy                           0.84        88
   macro avg       0.84      0.84      0.84        88
weighted avg       0.85      0.84      0.84        88

Confusion Matrix:
 [[31  5]
 [ 9 43]]


In [14]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(xgb_model, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)

best_xgb = grid_search.best_estimator_
pred = best_xgb.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))
print("MSE:", mean_squared_error(y_test, pred))
print("R²:", r2_score(y_test, pred))

model_lgb = lgb.LGBMRegressor(random_state=42)
model_lgb.fit(X_train, y_train)
pred_lgb = model_lgb.predict(X_test)
print("\nLightGBM Regression:")
print("MAE:", mean_absolute_error(y_test, pred_lgb))
print("MSE:", mean_squared_error(y_test, pred_lgb))
print("R²:", r2_score(y_test, pred_lgb))

model_svr = SVR()
model_svr.fit(X_train, y_train)
pred_svr = model_svr.predict(X_test)
print("\nSVR Regression:")
print("MAE:", mean_absolute_error(y_test, pred_svr))
print("MSE:", mean_squared_error(y_test, pred_svr))
print("R²:", r2_score(y_test, pred_svr))

model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model_nn.add(Dropout(0.2))
model_nn.add(Dense(32, activation='relu'))
model_nn.add(Dense(1))
model_nn.compile(optimizer='adam', loss='mean_squared_error')

history = model_nn.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)
pred_nn = model_nn.predict(X_test)
print("\nНейронная сеть Regression:")
print("MAE:", mean_absolute_error(y_test, pred_nn))
print("MSE:", mean_squared_error(y_test, pred_nn))
print("R²:", r2_score(y_test, pred_nn))

X_cls = daily_data[['headline_sentiment_mean', 'headline_sentiment_max',
                    'abstract_sentiment_mean', 'abstract_sentiment_max',
                    'headline_sentiment_count', 'abstract_sentiment_count',
                    'Close_Price_diff', 'Close_Price_diff_pct']]
y_cls = daily_data['price_increase_last']

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

model_xgb_cls = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model_xgb_cls.fit(X_train_cls, y_train_cls)
y_pred_xgb_cls = model_xgb_cls.predict(X_test_cls)
print("\nXGBoost Classifier:")
print("Accuracy:", accuracy_score(y_test_cls, y_pred_xgb_cls))
print(classification_report(y_test_cls, y_pred_xgb_cls))
print("Confusion Matrix:\n", confusion_matrix(y_test_cls, y_pred_xgb_cls))

# CatBoost Classifier
model_cat = CatBoostClassifier(random_state=42, verbose=0)
model_cat.fit(X_train_cls, y_train_cls)
y_pred_cat = model_cat.predict(X_test_cls)
print("\nCatBoost Classifier:")
print("Accuracy:", accuracy_score(y_test_cls, y_pred_cat))
print(classification_report(y_test_cls, y_pred_cat))
print("Confusion Matrix:\n", confusion_matrix(y_test_cls, y_pred_cat))


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters:  {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
MAE: 0.4924755394458771
MSE: 0.24904415011405945
R²: -0.0026677846908569336
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002483 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 44341
[LightGBM] [Info] Number of data points in the train set: 344, number of used features: 384
[LightGBM] [Info] Start training from score 0.552326
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4664 - val_loss: 0.3201
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2082 - val_loss: 0.2978
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1520 - val_loss: 0.3052
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1166 - val_loss: 0.3074
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0966 - val_loss: 0.3141
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0796 - val_loss: 0.2983
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0789 - val_loss: 0.3054
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0732 - val_loss: 0.3141
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0629 - val_loss: 0.3243
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0506 - val_loss: 0.3124
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0499 - val_loss: 0.3290
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0469 - val_lo

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:18:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



CatBoost Classifier:
Accuracy: 0.8409090909090909
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        40
           1       0.85      0.85      0.85        48

    accuracy                           0.84        88
   macro avg       0.84      0.84      0.84        88
weighted avg       0.84      0.84      0.84        88

Confusion Matrix:
 [[33  7]
 [ 7 41]]


In [17]:
#Новая модель, чтобы не запутаться в данных делаем все сначала
data = pd.read_csv('Apple_data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date', ascending=True)

def fill_missing_prices(data):
    data = data.reset_index(drop=True)
    for i in range(len(data)):
        if np.isnan(data.loc[i, 'Open Price']) and i > 0:
            data.loc[i, 'Open Price'] = data.loc[i - 1, 'Close Price']
        if np.isnan(data.loc[i, 'Close Price']) and i < len(data) - 1:
            data.loc[i, 'Close Price'] = data.loc[i + 1, 'Open Price']
    return data

data = fill_missing_prices(data)
data = data.dropna()

data['price_increase'] = (data['Close Price'] > data['Open Price']).astype(int)

bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
data['headline_embedding'] = data['headline'].apply(lambda x: bert_model.encode(x))

def aggregate_embeddings(embeddings_list):
    return np.mean(np.vstack(embeddings_list), axis=0)

daily_text = data.groupby('Date')['headline_embedding'].apply(list).reset_index()
daily_text['headline_embedding_avg'] = daily_text['headline_embedding'].apply(aggregate_embeddings)

embedding_dim = daily_text['headline_embedding_avg'].iloc[0].shape[0]
emb_cols = [f'emb_{i}' for i in range(embedding_dim)]
emb_df = pd.DataFrame(daily_text['headline_embedding_avg'].tolist(), columns=emb_cols)
daily_text = pd.concat([daily_text[['Date']], emb_df], axis=1)

daily_price = data.groupby('Date').agg({
    'Open Price': 'first',
    'Close Price': 'last'
}).reset_index()

daily_price['SMA_5'] = daily_price['Close Price'].rolling(window=5).mean()
daily_price['SMA_10'] = daily_price['Close Price'].rolling(window=10).mean()
daily_price['return'] = daily_price['Close Price'].pct_change()

daily_price = daily_price.dropna()

daily_merged = pd.merge(daily_price, daily_text, on='Date', how='inner')
daily_merged['price_increase'] = (daily_merged['Close Price'] > daily_merged['Open Price']).astype(int)

feature_cols = ['SMA_5', 'SMA_10', 'return'] + emb_cols
X = daily_merged[feature_cols]
y = daily_merged['price_increase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

estimators = [
    ('xgb', xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
    ('cat', CatBoostClassifier(random_state=42, verbose=0)),
    ('rf', RandomForestClassifier(random_state=42))
]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

y_pred = stacking_clf.predict(X_test)

print("Ensemble Stacking Classifier Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:25:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:27:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:27:28] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:27:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [21:27:31] WARNING: /w

Ensemble Stacking Classifier Results:
Accuracy: 0.7931034482758621
              precision    recall  f1-score   support

           0       0.75      0.82      0.79        40
           1       0.84      0.77      0.80        47

    accuracy                           0.79        87
   macro avg       0.79      0.80      0.79        87
weighted avg       0.80      0.79      0.79        87

Confusion Matrix:
 [[33  7]
 [11 36]]
